<h1> Exporing Data From the Google Play and App Store</h1>

<p> This project is to expore data from the Google Play Store and the App store and look for patterns.  Data are originally taken from Kaggle, I would include the information below. <p>

<br> Links:
<br> Google Play: https://www.kaggle.com/lava18/google-play-store-apps
<br> App Store: https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps



In [32]:
<p><b>Exploring and Cleaning Data</b></p>

SyntaxError: invalid syntax (<ipython-input-32-bfb3bcb05a45>, line 1)

In [ ]:
opened_file = open("googleplaystore.csv")
from csv import reader
read_file = reader(opened_file)
play_store_data = list(read_file)

opened_file = open("AppleStore.csv")
from csv import reader
read_file = reader(opened_file)
app_store_data = list(read_file)

#Function to show an overview of the datasets
def explore_data(dataset, start, end, rows_and_columns=True):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))


In [ ]:
# Exploring data from the Play Store
explore_data(play_store_data,0,3)

In [ ]:
# Exploring data from the App Store
explore_data(app_store_data,0,3)

It looks like the following columns may be helpful from the Play Store:
'App', 'Category', 'Rating', 'Reviews', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated'

And From the App Store:
'id', 'track_name', 'price', 'rating_count_tot', 'user_rating', 'prime_genre'

<p><b> Cleaning the Dataset </b></p>

In [ ]:
for i, row in enumerate(play_store_data[1:]):
    if float(row[2])>5 or float(row[2])<0:
        print ("Deleting row at index:", i)
        del play_store_data[i]
    else:
        row[2]=float(row[2])

<p>One row was deleted because the rating was incorrect.  Now we will check if there are duplicate names in the dataset, I will remove the one with less ratings (meaning that this data isn't as recent):</p>

In [ ]:
# Making sure that the number of ratings data is in int form
for i, row in enumerate(play_store_data):
    if row[3].isdigit()==False:
        print (i, row)

In [ ]:
#Changing this index position to have 3,000,000 ratings
play_store_data[10472][3]='3000000'
print(play_store_data[10472])

# Sorting by the largest number of ratings
play_store_data[1:].sort(key=lambda x: int(x[3]))

#Checked that data were sorted- but don't actually want to print them
# print(play_store_data[1])
# print(play_store_data.pop())

The following checks if an app name is in an app name set.  If it isn't, it adds
the name to the set.  If it is in the set, it skips this entry.  This works 
because the data were sorted above. Only the largest number of ratings will 
remain. While iterating, I also add the name of the app to a dictionary with the maximum number of ratings.

In [ ]:
#Used for the maximum number of reviews
reviews_max={}
app_name_set=set(play_store_data[0][0])
play_removed_duplicate_list=[play_store_data[0]]
length_before_removal= len(play_store_data)

while len(play_store_data)>1:
    app= play_store_data.pop()
    if app[0] in reviews_max:
        next
    
    else:
        app_name_set.add(app[0])
        play_removed_duplicate_list.append(app)
        reviews_max[app[0]]= app[3]
        
num_duplicates_removed= length_before_removal-len(play_removed_duplicate_list)
print("The follwing number of duplicates were removed:", num_duplicates_removed)
print("The length of the reviews_max dictionary is:", len(reviews_max))

The next step is to check for non-english apps.  I write a function to check if an app has any characters that are non-englist.  I will try a few tests below as well to make sure my function works as I expect.

In [ ]:
def check_all_eng_char(check_string):
    i=0
    for letter in check_string:
        if ord(letter)>127:
            i+=1
            if i>3:
                return False
    return True

#'Instagram' Expect True:
print(check_all_eng_char('Instagram'))

# '爱奇艺PPS -《欢乐颂2》电视剧热播': Expect False
print(check_all_eng_char('爱奇艺PPS -《欢乐颂2》电视剧热播'))

# 'Docs To Go™ Free Office Suite': Expect True- only one character
print(check_all_eng_char('Docs To Go™ Free Office Suite'))

# 'Instachat 😜': Expect True- only one character
print(check_all_eng_char('Instachat 😜'))


In the google play data, how many rows remain if we remove the non-english apps?

In [ ]:
eng_apps_play=[]
eng_apps_appstore=[]
num_eng_apps=0
for app in play_removed_duplicate_list:
    if check_all_eng_char(app[0])==True:
        eng_apps_play.append(app)
        num_eng_apps+=1
print("There are this number of apps with Eng titles in the google play store:", num_eng_apps)

appstore_num_eng_apps=0
for app in app_store_data:
    if check_all_eng_char(app[1])==True:
        eng_apps_appstore.append(app)
        appstore_num_eng_apps+=1
        
print("There are this number of apps with Eng titles in the app store:", appstore_num_eng_apps)


Removing Non-free apps: We only wish to explore free apps, the next step will be to remove all non-free apps from each dataset:
    

In [ ]:
free_eng_apps_play=[eng_apps_play[1]]
free_eng_apps_appstore=[eng_apps_appstore[1]]

for app in eng_apps_play[1:]:
    if app[7]=='0':
        free_eng_apps_play.append(app)
print("Play: there are this num of apps that are in english and free:", (len(free_eng_apps_play)-1))

for app in eng_apps_appstore[1:]:
      if app[4]=='0.0':
        free_eng_apps_appstore.append(app)
print("Appstore: there are this num of apps that are in english and free:", (len(free_eng_apps_appstore)-1))



<h2> Data Analysis</h2>

We wish to explore which apps are most popular, and will start by generating a frequency table by genera.

In [ ]:
def generate_freq_tbl(a_data_set, gen_idx):
    frequency_table = {}
    table_display = []
    
    for row in a_data_set[1:]:
        a_data_point = row[gen_idx]
        if a_data_point in frequency_table:
            frequency_table[a_data_point] += 1
        else:
            frequency_table[a_data_point] = 1
    for key in frequency_table.keys():
        frequency_table[key]= frequency_table[key]/(len(a_data_set)-1)
        key_val_as_tuple = (frequency_table[key], key)
        table_display.append(key_val_as_tuple)
    
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted[:6]:
        print(entry[1], ':', entry[0])
        
    return frequency_table

print("Play Store Catagories")
play_freq_tbl_cat= generate_freq_tbl(free_eng_apps_play, 1)

print("")
print("Play Store Genera")
play_freq_tbl_gen= generate_freq_tbl(free_eng_apps_play, 9)

print("")
print("AppStore Genera")
app_freq_tbl= generate_freq_tbl(free_eng_apps_appstore, 11)

<h2> Analysis of Frequency Tables</h2>

<h4>AppStore Analysis</h4>
The most common genera in the AppStore were "Games"(58%), "Entertainment" (8%) and "Photo and Video" (5%).  Catagories that were more entertaining seemed to dominate in populated in the AppStore.  We can't tell how much these apps were used, and if they were highly reviewed.

<h4>PlayStore Analysis</h4>
The PlayStore has two different catagories that could be considered "generas" the "Catagories" and "Genera" options.  "Family" apps seem to dominate the PlayStore
The most common genera in the AppStore were "Games"(58%), "Entertainment" (8%) and "Photo and Video" (5%).  Catagories that were more entertaining seemed to dominate in populated in the AppStore.  We can't tell how much these apps were used, and if they were highly reviewed.

In [ ]:
for genre in app_freq_tbl:
    total = 0
    len_genre = 0
    for app in free_eng_apps_appstore:
        genre_app = app[-5]
        if genre_app == genre:            
            n_ratings = float(app[5])
            total += n_ratings
            len_genre += 1
    avg_n_ratings = total / len_genre
    print(genre, ':', avg_n_ratings)

Navigation has the highest number of reviews, followed by reference apps.  More popular app types could have an oversaturation of the market, and it may be helpful to produce an app in a genere that is growing.

In [33]:
for catagory in play_freq_tbl_cat:
    total = 0
    len_catagory = 0
    for app in free_eng_apps_play:
        catagory_app = app[1]
        if catagory_app==catagory:
            n_ratings = app[5].replace("+","")
            n_ratings=n_ratings.replace(",","")
            total += float(n_ratings)
            len_catagory += 1
    avg_n_ratings = total / len_catagory
    print(catagory, ':', avg_n_ratings)
            

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1704192.3399014778
COMICS : 817657.2727272727
COMMUNICATION : 38326063.197916664
DATING : 854028.8303030303
EDUCATION : 1776262.6262626264
ENTERTAINMENT : 9200779.220779222
EVENTS : 253542.22222222222
FAMILY : 5180430.984182777
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
GAME : 12917897.209026128
HEALTH_AND_FITNESS : 4167457.3602941176
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1462491.1498559078
MAPS_AND_NAVIGATION : 4056941.7741935486
MEDICAL : 123064.7898089172
NEWS_AND_MAGAZINES : 9549178.467741935
PARENTING : 542603.6206896552
PERSONALIZATION : 5218893.815699658
PHOTOGRAPHY : 17840110.40229885
PRODUCTIVITY : 16772838.591304347
SHOPPING : 7036877.311557789
SOCIAL : 23253652.127118643
SPORTS : 4274688.722772277
TOOLS : 10801391.298666667
TRAVEL_AND_LOCAL : 13984077.71014

Communications apps have the largest number of installations.  It looks like the data may be heavily skewed towards apps that have over 1M installations such as WhatsApp.  We haven't analized ratings of apps.  It would be helpful in the future to understand if free apps are more highly rated, and how there may be room in the market of less popular catagories for apps.